In [2]:
from typing import Callable, Tuple, List
import torch
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import os
from utils import train_transform,test_transform

def create_dataloader(
    train_path: str,
    test_path: str,
    train_transform: Callable,
    test_transform: Callable,
    batch_size: int = 32,
    num_workers: int = os.cpu_count() or 1,
    pin_memory: bool = True
) -> Tuple[DataLoader, DataLoader, List[str]]:
    """
    Creates training and testing DataLoaders.
    
    Args:
        train_path: Path to training data folder.
        test_path: Path to testing data folder.
        train_transform: Function returning transforms for training data.
        test_transform: Function returning transforms for testing data.
        batch_size: Number of samples per batch.
        num_workers: Number of subprocesses for data loading.
        pin_memory: Whether to pin memory (useful for GPU training).
    
    Returns:
        Tuple of (train_dataloader, test_dataloader, class_names)
    """
    # Get transforms
    train_transforms = train_transform()
    test_transforms = test_transform()
    
    # Load datasets
    train_data = ImageFolder(root=train_path, transform=train_transforms)
    test_data = ImageFolder(root=test_path, transform=test_transforms)
    
    class_names = train_data.classes
    
    # Create DataLoaders
    train_dataloader = DataLoader(
        dataset=train_data,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=pin_memory
    )
    
    test_dataloader = DataLoader(
        dataset=test_data,
        batch_size=batch_size,
        shuffle=False,
        num_workers=num_workers,
        pin_memory=pin_memory
    )
    
    return train_dataloader, test_dataloader, class_names


In [4]:
train_dataloader,test_dataloader,class_names = create_dataloader(train_path="./data_split/train",
                  test_path="./data_split/test",
                train_transform=train_transform,
                test_transform=test_transform,
                batch_size=32,
                pin_memory=True)

In [6]:
len(class_names)

34

In [7]:
next(iter(train_dataloader))

[tensor([[[[ 1.8379,  1.7352,  1.7009,  ...,  0.7419,  0.8789,  0.9988],
           [ 1.6667,  1.6838,  1.6838,  ...,  0.7077,  0.7933,  0.8447],
           [ 1.5982,  1.6667,  1.6838,  ...,  0.7419,  0.7933,  0.8104],
           ...,
           [ 0.4851,  0.5022,  0.6049,  ..., -0.7308, -1.1418, -1.1932],
           [ 0.3994,  0.3994,  0.4508,  ..., -0.8678, -1.0048, -1.3473],
           [ 0.3481,  0.2967,  0.2453,  ..., -1.1075, -1.0562, -1.3644]],
 
          [[ 1.8158,  1.7108,  1.6758,  ...,  0.5378,  0.6779,  0.8004],
           [ 1.6408,  1.6583,  1.6583,  ...,  0.5028,  0.5903,  0.6429],
           [ 1.5707,  1.6408,  1.6583,  ...,  0.5378,  0.5903,  0.6078],
           ...,
           [ 0.4153,  0.4328,  0.5378,  ..., -0.8102, -1.2479, -1.3004],
           [ 0.3277,  0.3277,  0.3803,  ..., -0.9503, -1.1078, -1.4580],
           [ 0.2752,  0.2227,  0.1702,  ..., -1.1954, -1.1604, -1.4755]],
 
          [[ 1.9254,  1.8208,  1.7860,  ...,  0.7402,  0.8797,  1.0017],
           [ 

In [8]:
from base_model import create_base_model

base_model,num_features = create_base_model()

Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\ANAND REDDY/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth
100%|██████████| 20.5M/20.5M [00:11<00:00, 1.89MB/s]


In [9]:
base_model

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [19]:
from torchinfo import summary

summary(
    model=base_model,
    input_size=(1, 3, 224, 224),  # This works with torchinfo
    col_names=["input_size", "output_size", "num_params","trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                                      Input Shape          Output Shape         Param #              Trainable
EfficientNet (EfficientNet)                                  [1, 3, 224, 224]     [1, 1280]            --                   True
├─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   True
│    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    --                   True
│    │    └─Conv2d (0)                                       [1, 3, 224, 224]     [1, 32, 112, 112]    864                  True
│    │    └─BatchNorm2d (1)                                  [1, 32, 112, 112]    [1, 32, 112, 112]    64                   True
│    │    └─SiLU (2)                                         [1, 32, 112, 112]    [1, 32, 112, 112]    --                   --
│    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16, 112,

In [9]:
from torchinfo import summary
from transfer_learning import create_tranfer_learning_model
model = create_tranfer_learning_model()
summary(
    model=model,
    input_size=(1, 3, 224, 224),  
    col_names=["input_size", "output_size", "num_params","trainable"],
    col_width=20,
    row_settings=["var_names"]
)

Layer (type (var_name))                                           Input Shape          Output Shape         Param #              Trainable
Sequential (Sequential)                                           [1, 3, 224, 224]     [1, 34]              --                   Partial
├─EfficientNet (0)                                                [1, 3, 224, 224]     [1, 1280]            --                   False
│    └─Sequential (features)                                      [1, 3, 224, 224]     [1, 1280, 7, 7]      --                   False
│    │    └─Conv2dNormActivation (0)                              [1, 3, 224, 224]     [1, 32, 112, 112]    (928)                False
│    │    └─Sequential (1)                                        [1, 32, 112, 112]    [1, 16, 112, 112]    (1,448)              False
│    │    └─Sequential (2)                                        [1, 16, 112, 112]    [1, 24, 56, 56]      (16,714)             False
│    │    └─Sequential (3)                       

In [10]:
!pip install torchmetrics

   ---------------------------------------- 0.0/983.0 kB ? eta -:--:--
   ---------- ----------------------------- 262.1/983.0 kB ? eta -:--:--
   -------------------------------- ------- 786.4/983.0 kB 2.4 MB/s eta 0:00:01
   ---------------------------------------- 983.0/983.0 kB 2.2 MB/s eta 0:00:00
